# Milestone 1

## Import

In [1]:
import json
import os
import requests
import pandas as pd
import zipfile

## Download the Data

### Download from API

In [2]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
output_dir = "../data/"

In [3]:
resp = requests.get(url)

file_list = json.loads(resp.content)['files']

Note: The cell below takes around 1.5 minutes to run. 

In [4]:
%%time
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
for file in file_list:
    if file['name'] != "data.zip":
        continue
    output_name = os.path.join(output_dir, file['name'])
    if os.path.exists(output_name):
        print(f"{file['name']} already exists!")
    else:
        print(f"Downloading {file['name']} to {output_dir}...")
        file_resp = requests.get(file['download_url'])
        with open(output_name, "wb") as f:
            for chunk in file_resp.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
        print("Download finished")

data.zip already exists!
CPU times: user 615 µs, sys: 849 µs, total: 1.46 ms
Wall time: 1.71 ms


### Unzip data

In [5]:
raw_csv_dir = os.path.join(output_dir, "raw")

In [6]:
%%time
with zipfile.ZipFile(output_name, "r") as zf:
    zf.extractall(raw_csv_dir)

CPU times: user 21.5 s, sys: 6.79 s, total: 28.3 s
Wall time: 39.8 s


## Combine CSV Files

In [7]:
dir_files = os.listdir(raw_csv_dir)
output_df = pd.DataFrame()

In [8]:
%%time
# Exclude MACOSX_ & observed_daily_rainfall_SYD.csv
for fname in dir_files:
    if fname[-8:] != "_NSW.csv":
        continue

    model_name = fname.split('_')[0]

    df = pd.read_csv(os.path.join(raw_csv_dir, fname), index_col=0)
    df['model'] = model_name
    output_df = pd.concat([output_df, df])
    del df

output_df.to_csv(os.path.join(output_dir, "combined.csv"))

CPU times: user 8min 11s, sys: 1min 35s, total: 9min 47s
Wall time: 10min 39s


In [9]:
%%sh
du -sh ../data/combined.csv

5.6G	../data/combined.csv


In [10]:
output_df.shape

(62467843, 6)

In [11]:
output_df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [12]:
del output_df

### Comparison

| Team Member| Operating System | RAM        | Processor                        | Is SSD | CPU Time   | Wall Time |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    | 7min 54s   | 8min 10s  |
| Kyle       |Windows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |      | 8min 15s  |
| LG         | Big Sur 11.5.2  | 8 GB | 2.9 GHz Dual-Core Intel Core i5 |  Yes      |  9min 47s    | 10min 39s  | 
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    | 6min 33s   | 6min 34s  |

**Discussion:**
> The running time among the team members are roughly the same. However, we could not see a clear relationship between the RAM/CPU with the running time. In reality, there could be many external conditions affecting the running time such as number of process running concurrently. The memory management algorithm among various operating systems might also be accountable. Note that `%%time` could not report the CPU time on Windows.

## Load the Combined CSV and simple EDA

In [13]:
%reset -f

In [14]:
import pandas as pd

### Approach 1: Load Only Columns We want

Let's try `value_counts()` on `model` column. Let's compare how long it takes when we only load in `model` column vs all columns.

In [15]:
%%time
use_cols = ["model"]
df = pd.read_csv("../data/combined.csv", usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 34.3 s, sys: 2.93 s, total: 37.2 s
Wall time: 39.1 s


In [16]:
%%time
df = pd.read_csv("../data/combined.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 1min 10s, sys: 25.5 s, total: 1min 36s
Wall time: 1min 51s


#### Comparison - Load Only Columns We Want

| Team Member| Operating System | RAM        | Processor        | Is SSD | CPU Time (model)   | Wall Time (model) |  CPU Time (all) | Wall Time (all) |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|:---------------:|:---------------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    |    32.7 s  |   33.1 s  |   1min 31s      | 1min 48s        |
| Kyle       | Windows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |            | 46.5s|     |1min 23s
| LG         | Big Sur 11.5.2 | 8 GB | 2.9 GHz Dual-Core Intel Core i5 | Yes | 37.2 s | 39.1 s     | 1min 36s     | 1min 51s
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    |  29.2s     | 29.7s| 1min 03s      | 1min 03s

**Discussion:**
> We can observe that when only `model` column is loaded, there is a significant reduction in Wall Time.

### Approach 2

In [17]:
%reset -f
import pandas as pd

In [18]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("../data/combined.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
CPU times: user 1min 9s, sys: 9.32 s, total: 1min 19s
Wall time: 1min 24s


#### Comparison - Load in Chunk

| Team Member| Operating System | RAM        | Processor        | Is SSD | CPU Time (chunk)   | Wall Time (chunk) |  CPU Time (all) | Wall Time (all) |
|:----------:|:----------------:|:----------:|:--------------------------------:|:------:|:----------:|:---------:|:---------------:|:---------------:|
| James      | Big Sur 11.5     | 8 GB DDR3  | 1.4 GHz Quad-Core Intel Core i5  | Yes    |    1min 8s |  1min 10s |   1min 31s      | 1min 48s        |
| Kyle       | Windows 11       | 32 GB      | 2.2 GHz Hexa-Core Intel i7-8750H | Yes    |            | 1min 14s|     |1min 23s
| LG         | Big Sur 11.5.2 | 8 GB  | 2.9 GHz Dual-Core Intel Core i5 | Yes | 1min 19s           | 1min 24s    | 1min 36s | 1min 51s
| Philson    | Ubuntu 20.04     | 16 GB DDR4 | 1.8GHz Quad-Core Intel i7-8565U  | Yes    |  1min          | 1min 01s    |  1min 03s    | 1min 03s

**Discussion:**
> We can observe that there is a reasonable reduction in Wall Time when we run `value_counts()` by loading in smaller chunks, 10 million rows each in our case. It seems that there is not a significant reduction in time because RAM size is large enough to load in all the data in all.

## Simple EDA in R

In [3]:
%reset -f
%load_ext rpy2.ipython

In [4]:
%%R
library(dplyr)
library(arrow)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




### 6.1 Using arrow table and pyra 

In [6]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [7]:
%%time
# loading csv file to pyarrow dataset and then converting to pyarrow table
dataset = ds.dataset("../data/combined.csv", format="csv")
table = dataset.to_table()

CPU times: user 26.7 s, sys: 4.97 s, total: 31.7 s
Wall time: 29.3 s


In [8]:
%%time
# converting pyarrow table to rarrow table
r_table = pyra.converter.py2rpy(table)

CPU times: user 38 s, sys: 706 ms, total: 38.7 s
Wall time: 42.6 s


In [9]:
%%time
%%R -i r_table
# referenced lecture 2 note
# performing simple EDA to count the number of each model
start_time <- Sys.time()
library(dplyr)
counts <- r_table %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 9.600725 secs
CPU times: user 6.36 s, sys: 3.17 s, total: 9.53 s
Wall time: 9.73 s


### 6.2 Using feather

In [10]:
%%time
# creating feather file from above table
import pyarrow.feather as feather
feather.write_feather(table, '../data/combined.feather')

CPU times: user 5.5 s, sys: 10.4 s, total: 15.9 s
Wall time: 11.8 s


In [11]:
%%time
%%R
# transfering the feather file to R
r_table_feather <- arrow::read_feather("../data/combined.feather")

CPU times: user 5.03 s, sys: 8.25 s, total: 13.3 s
Wall time: 13 s


In [ ]:
%%time
%%R
# performing the same EDA as above
start_time <- Sys.time()
library(dplyr)
counts <- r_table_feather %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

### Discussion
We chose to experiment with Feather with the following reasons:  
- According to various sources, Feather is faster than Parquet to read into Arrow Table.
- Compared to vanila serialization/deserialization process as performed above, Feather indeed performed much faster.
- Using Feather file also saved the storage, as the file generated is only 1.1 GB.
- One reservation is that Feather is not recommended for long-term data storage.